# Import Data

In [2]:
import pandas as pd

In [5]:
import warnings
warnings.filterwarnings('ignore')

When we created database of data downloaded from google cloud platform, using code like

SELECT name1_new

FROM `bigquery-public-data.covid19_open_data.covid19_open_data`

WHERE country_code = 'US'

AND date BETWEEN '2020-03-01' AND '2021-11-10'


We read in database in python, and then use sql code as below to clean the data and store it to new variable searchtrend.

# Process Geographical data

In [29]:
data2 = pd.read_csv("data/covid_demographic.csv")

In [30]:
data2_sum_df = data2.groupby(['subregion1_code','date'])['area_sq_km', 'area_urban_sq_km', 'population',
                                                         'population_age_60_69', 'population_age_70_79',
                                                         'population_age_80_and_older'].sum().reset_index()

In [31]:
data2_sum_df['population_den'] = data2_sum_df['population']/data2_sum_df['area_sq_km']

In [32]:
data2_sum_df['elder_perc'] = (data2_sum_df['population_age_60_69'] + data2_sum_df['population_age_70_79'] + 
                              data2_sum_df['population_age_80_and_older']) /data2_sum_df['population']

In [33]:
data2_avg_df = data2.groupby(['subregion1_code','date'])['average_temperature_celsius',
                                                         'mobility_retail_and_recreation',
                                          'mobility_grocery_and_pharmacy', 'mobility_parks',
                                          'mobility_transit_stations', 'mobility_workplaces',
                                          'mobility_residential'
                                         ].mean().reset_index()

In [34]:
geo_df = pd.merge(data2_sum_df, data2_avg_df, on = ['subregion1_code', 'date'])

In [35]:
geo_df = geo_df.drop(['area_sq_km', 'area_urban_sq_km','population', 
                      'population_age_60_69', 'population_age_70_79', 'population_age_80_and_older'], 1)

In [36]:
geo_df.to_csv('data/geo.csv', index = False)

# Process policy data

In [14]:
df = pd.read_csv("data/covid_policy.csv")

df = df[df.subregion1_code != None]

policy = df.drop(columns = ['emergency_investment_in_healthcare', 'investment_in_vaccines'])

In [15]:
new_policy = policy.groupby(by = ['date', 'country_code', 'subregion1_code']).mean().reset_index()

new_policy.to_csv("data/policy.csv", index = False)

In [ ]:
", ".join([f"AVG({ele}) AS {ele}" for ele in df.columns[3:]])

# Process search trend data

In [3]:
data4 = pd.read_csv("data/covid_searchtrend.csv")
# originally we have 57 subregions including Nan subregion, it has 602 rows, we delete observations with Nan subregion
data4 = data4.dropna(subset=["subregion1_code"])

In [17]:
# group the data by subregion and date first, get mean in same subregion and date
st = data4.groupby(['subregion1_code','date']).mean().reset_index()

In [18]:

# after imputation drop the columns still have more than 100,000 missing values

st2 = st.drop(["search_trends_acute_bronchitis",
                    "search_trends_ageusia", 
                    "search_trends_anosmia", 
                    "search_trends_aphasia",
                    "search_trends_aphonia",
                    "search_trends_asphyxia",
                    "search_trends_chills",
                    "search_trends_clouding_of_consciousness"], axis =1)

In [19]:
st2.to_csv("data/search_trend.csv", index = False)

# Process case data

In [145]:
case = pd.read_csv('data/covid_case.csv')
data_case = case.groupby(['subregion1_code','date'])['new_confirmed', 'new_tested',
                                                    'cumulative_confirmed', 'cumulative_tested',
                                                    'cumulative_recovered'].sum().reset_index()
data_case['cumulative_confirmed'][data_case['cumulative_confirmed'] < 0] = 0
sat_list = pd.date_range(start='2020-02-29', end='2021-11-06', freq='7D')
sat_list = sat_list.strftime('%Y-%m-%d').tolist()
sun_list = pd.date_range(start='2020-03-01', end='2021-11-07', freq='7D')
sun_list = sun_list.strftime('%Y-%m-%d').tolist()
weekend = sat_list + sun_list
data_case = data_case.sort_values(by=['subregion1_code', 'date'])
data_case = data_case[data_case.date != '2020-03-01']

data_case['weekly_new_avg'] = data_case.groupby(data_case.index // 5)['new_confirmed'].transform('mean')

data_case['new_confirmed'][data_case.date.isin(weekend) & (data_case.new_confirmed == 0)] = data_case['weekly_new_avg']
data_case = data_case.drop(['new_tested', 'weekly_new_avg','cumulative_tested'], 1)
data_case.to_csv('data/case.csv', index = False)

<ipython-input-145-9d9393c30086>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_case = case.groupby(['subregion1_code','date'])['new_confirmed', 'new_tested',
<ipython-input-145-9d9393c30086>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_case['cumulative_confirmed'][data_case['cumulative_confirmed'] < 0] = 0


In [94]:
data_case

,subregion1_code,date,new_confirmed,cumulative_confirmed,cumulative_recovered
1,AK,2020-03-02,0.0,0.0,0.0
2,AK,2020-03-03,0.0,0.0,0.0
3,AK,2020-03-04,0.0,0.0,0.0
4,AK,2020-03-05,0.0,0.0,0.0
5,AK,2020-03-06,0.0,0.0,0.0
...,...,...,...,...,...
34715,WY,2021-11-06,552.0,210636.0,0.0
34716,WY,2021-11-07,552.0,210636.0,0.0
34717,WY,2021-11-08,1344.0,211980.0,0.0
34718,WY,2021-11-09,594.0,212574.0,0.0


# Process vaccination and test data

In [146]:
data5 = pd.read_csv("data/states_timeseries.csv")
data5 = data5[['date','state', 'actuals.positiveTests', 'actuals.negativeTests', 'actuals.vaccinationsCompleted']]
data5['cumulative_tested'] = data5['actuals.positiveTests'] + data5['actuals.negativeTests']
data5 = data5.drop(['actuals.negativeTests','actuals.positiveTests'], 1)
data5 = data5.rename(columns={'state': 'subregion1_code', 'actuals.vaccinationsCompleted': 'vaccination_count'})
data5.to_csv('test_vaccination.csv', index = False)

# Join 4 df

In [154]:
geo = pd.read_csv('data/geo.csv')
policy = pd.read_csv('data/policy.csv')
search_trend = pd.read_csv('data/search_trend.csv')
case = pd.read_csv('data/case.csv')
test_vaccination = pd.read_csv('test_vaccination.csv')

df = pd.merge(geo, policy, on = ['subregion1_code', 'date'])
df = pd.merge(df, search_trend, on = ['subregion1_code', 'date'])
df = pd.merge(df, case, on = ['subregion1_code', 'date'])
#df = pd.merge(df, test_vaccination, on = ['subregion1_code', 'date'])

df = df.drop(['country_code','cumulative_recovered'], 1)

#df = df.dropna()
#df.to_csv('data/full_data.csv')

In [148]:
len(df.date.unique())

614

In [149]:
df['date'] =pd.to_datetime(df['date'])
df = df.sort_values(by='date')

In [150]:
dates = df['date']
dates = dates.drop_duplicates()
test_date = dates[-21:]
test_df = df[df.date.isin(test_date)]
train_df = df[~df.date.isin(test_date)]

train_df['date'] = train_df['date'].dt.strftime('%Y-%m-%d')
test_df['date'] = test_df['date'].dt.strftime('%Y-%m-%d')

<ipython-input-150-1e258e62481a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['date'] = train_df['date'].dt.strftime('%Y-%m-%d')
<ipython-input-150-1e258e62481a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['date'] = test_df['date'].dt.strftime('%Y-%m-%d')


In [142]:
train_df.to_csv('data/train_df.csv', index = False)
test_df.to_csv('data/test_df.csv', index = False)

In [48]:
df.isna().sum()

subregion1_code                   0
date                              0
population_den                    0
elder_perc                        0
average_temperature_celsius       0
mobility_retail_and_recreation    0
mobility_grocery_and_pharmacy     0
mobility_parks                    0
mobility_transit_stations         0
mobility_workplaces               0
mobility_residential              0
school_closing                    0
workplace_closing                 0
restrictions_on_gatherings        0
public_transport_closing          0
stay_at_home_requirements         0
public_information_campaigns      0
testing_policy                    0
facial_coverings                  0
vaccination_policy                0
stringency_index                  0
search_trends_anxiety             0
search_trends_apnea               0
search_trends_asthma              0
search_trends_chest_pain          0
search_trends_cough               0
search_trends_fatigue             0
search_trends_diarrhea      

In [64]:
train_df.to_csv('train_df.csv', index = False)
test_df.to_csv('test_df.csv', index = False)

# Load into SQL

In [ ]:
from pathlib import Path
Path('covid_project.db').touch()

In [ ]:
import sqlite3
conn = sqlite3.connect('covid_project.db')
c = conn.cursor()

In [ ]:
data1 = pd.read_csv("data/covid_case.csv")
data1.to_sql('case', conn, if_exists = 'append', index = False)

data2 = pd.read_csv("data/covid_demographic.csv")
data2.to_sql('demographic', conn, if_exists = 'append', index = False)

data3 = pd.read_csv("data/covid_policy.csv")
data3.to_sql('policy', conn, if_exists = 'append', index = False)

data4 = pd.read_csv("data/covid_searchtrend.csv")
data4.to_sql('searchtrend', conn, if_exists = 'append', index = False)

In [8]:
%load_ext sql

In [12]:
%sql sqlite:///covid_project.db

In [13]:
%%sql

SELECT * FROM sqlite_master WHERE type = 'table';

 * sqlite:///covid_project.db
Done.


type,name,tbl_name,rootpage,sql
table,case,case,2,"CREATE TABLE ""case"" (""date"" TEXT, ""country_code"" TEXT, ""subregion1_code"" TEXT, ""new_confirmed"" REAL, ""new_deceased"" REAL, ""new_tested"" REAL, ""cumulative_confirmed"" REAL, ""cumulative_tested"" REAL, ""cumulative_recovered"" REAL, ""cumulative_hospitalized_patients"" REAL)"
table,demographic,demographic,16840,"CREATE TABLE ""demographic"" (""date"" TEXT, ""country_code"" TEXT, ""subregion1_code"" TEXT, ""area_sq_km"" INTEGER, ""area_urban_sq_km"" REAL, ""average_temperature_celsius"" REAL, ""population"" INTEGER, ""population_age_60_69"" REAL, ""population_age_70_79"" REAL, ""population_age_80_and_older"" REAL, ""life_expectancy"" REAL, ""nurses_per_1000"" REAL, ""physicians_per_1000"" REAL, ""health_expenditure_usd"" REAL, ""population_clustered"" REAL, ""human_capital_index"" REAL, ""comorbidity_mortality_rate"" REAL, ""mobility_retail_and_recreation"" REAL, ""mobility_grocery_and_pharmacy"" REAL, ""mobility_parks"" REAL, ""mobility_transit_stations"" REAL, ""mobility_workplaces"" REAL, ""mobility_residential"" REAL)"
table,policy,policy,53201,"CREATE TABLE ""policy"" (""date"" TEXT, ""country_code"" TEXT, ""subregion1_code"" TEXT, ""school_closing"" REAL, ""workplace_closing"" REAL, ""restrictions_on_gatherings"" REAL, ""public_transport_closing"" REAL, ""stay_at_home_requirements"" REAL, ""public_information_campaigns"" REAL, ""testing_policy"" REAL, ""emergency_investment_in_healthcare"" REAL, ""investment_in_vaccines"" REAL, ""facial_coverings"" REAL, ""vaccination_policy"" REAL, ""stringency_index"" REAL)"
table,searchtrend,searchtrend,71369,"CREATE TABLE ""searchtrend"" (""date"" TEXT, ""country_code"" TEXT, ""subregion1_code"" TEXT, ""search_trends_acute_bronchitis"" REAL, ""search_trends_ageusia"" REAL, ""search_trends_anosmia"" REAL, ""search_trends_anxiety"" REAL, ""search_trends_aphasia"" REAL, ""search_trends_aphonia"" REAL, ""search_trends_apnea"" REAL, ""search_trends_asphyxia"" REAL, ""search_trends_asthma"" REAL, ""search_trends_chest_pain"" REAL, ""search_trends_chills"" REAL, ""search_trends_clouding_of_consciousness"" REAL, ""search_trends_cough"" REAL, ""search_trends_fatigue"" REAL, ""search_trends_diarrhea"" REAL, ""search_trends_headache"" REAL, ""search_trends_heart_arrhythmia"" REAL, ""search_trends_fever"" REAL, ""search_trends_common_cold"" REAL, ""search_trends_pain"" REAL, ""search_trends_nausea"" REAL, ""search_trends_sore_throat"" REAL, ""search_trends_swelling"" REAL, ""search_trends_vomiting"" REAL)"


## process policy data

In [ ]:
%%sql policy <<

SELECT date, country_code, subregion1_code, 
AVG(school_closing) AS school_closing, 
AVG(workplace_closing) AS workplace_closing, AVG(restrictions_on_gatherings) AS restrictions_on_gatherings, AVG(public_transport_closing) AS public_transport_closing, AVG(stay_at_home_requirements) AS stay_at_home_requirements, AVG(public_information_campaigns) AS public_information_campaigns, AVG(testing_policy) AS testing_policy, AVG(emergency_investment_in_healthcare) AS emergency_investment_in_healthcare, AVG(investment_in_vaccines) AS investment_in_vaccines, AVG(facial_coverings) AS facial_coverings, AVG(vaccination_policy) AS vaccination_policy, AVG(stringency_index) AS stringency_index
FROM policy
WHERE subregion1_code IS NOT NULL
GROUP BY date, country_code, subregion1_code;

## process search trend data

In [ ]:
%%sql searchtrend <<

SELECT date, country_code, subregion1_code,
AVG(search_trends_apnea) AS search_trends_apnea, 
AVG(search_trends_asthma) AS search_trends_asthma, 
AVG(search_trends_chest_pain) AS search_trends_chest_pain, 
AVG(search_trends_cough) AS search_trends_cough, 
AVG(search_trends_fatigue) AS search_trends_fatigue, 
AVG(search_trends_diarrhea) AS search_trends_diarrhea, 
AVG(search_trends_headache) AS search_trends_headache, 
AVG(search_trends_heart_arrhythmia) AS search_trends_heart_arrhythmia, 
AVG(search_trends_fever) AS search_trends_fever, 
AVG(search_trends_common_cold) AS search_trends_common_cold, 
AVG(search_trends_pain) AS search_trends_pain, 
AVG(search_trends_nausea) AS search_trends_nausea, 
AVG(search_trends_sore_throat) AS search_trends_sore_throat, 
AVG(search_trends_swelling) AS search_trends_swelling, 
AVG(search_trends_vomiting) AS search_trends_vomiting

FROM searchtrend
WHERE subregion1_code IS NOT NULL
GROUP BY date, subregion1_code

## process geographical data

In [ ]:
%% sql geo <<
SELECT date, country_code, subregion1_code,
SUM(area_sq_km) AS area_sq_km, 
AVG(average_temperature_celsius) AS average_temperature_celsius, 
SUM(population) AS population, 
SUM(population_age_60_69) AS population_age_60_69, 
SUM(population_age_70_79) AS population_age_70_79, 
SUM(population_age_80_and_older) AS population_age_80_and_older, 
AVG(life_expectancy) AS life_expectancy, 
AVG(health_expenditure_usd) AS health_expenditure_usd, 
AVG(population_clustered) AS population_clustered, 
AVG(human_capital_index) AS human_capital_index, 
AVG(comorbidity_mortality_rate) AS comorbidity_mortality_rate, 
AVG(mobility_retail_and_recreation) AS mobility_retail_and_recreation, 
AVG(mobility_grocery_and_pharmacy) AS mobility_grocery_and_pharmacy, 
AVG(mobility_parks) AS mobility_parks, 
AVG(mobility_transit_stations) AS mobility_transit_stations, 
AVG(mobility_workplaces) AS mobility_workplaces, 
AVG(mobility_residential) AS mobility_residential
FROM demographic
GROUP BY date, country_code, subregion1_code

In [85]:
data5

,date,state,actuals.vaccinationsCompleted,cumulative_tested
0,2020-03-01,AK,NaN,NaN
1,2020-03-02,AK,NaN,NaN
2,2020-03-03,AK,NaN,NaN
3,2020-03-04,AK,NaN,NaN
4,2020-03-05,AK,NaN,NaN
...,...,...,...,...
33667,2021-11-19,WY,261342.0,1144652.0
33668,2021-11-20,WY,261698.0,NaN
33669,2021-11-21,WY,261735.0,NaN
33670,2021-11-22,WY,261758.0,NaN
